# Creating the Channel Funding Transaction

In this section, we’ll create a Lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.


## Setup

For this exercise we'll need Bitcoin Core. This notebook has been tested with v29.0

Below, set the paths for:

    The bitcoin core functional test framework directory.
    The directory containing taproot-lightning-channels-workshop.

You'll need to edit these next two lines for your local setup.


In [1]:
path_to_bitcoin_functional_test = "/home/pins-dev/Projects/bitcoin/build/test/functional"
path_to_taproot_workshop = "/home/pins-dev/Projects/Taproot-Lightning-Channels-Workshop"


### Setup bitcoin core test framework

Start up regtest mode, delete any regtest network history so we are starting from scratch.

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_taproot_workshop)
from functions import *

import json

# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

# Create a new wallet and address to send mining rewards so we can fund our transactions
node.createwallet(wallet_name='mywallet')
address = node.getnewaddress()

# Generate 101 blocks so that the first block subsidy reaches maturity
result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)

# Check that we were able to mine 101 blocks
assert(node.getblockcount() == 101)

2025-08-18T02:00:00.102000Z TestFramework (INFO): PRNG seed is: 2878485877312614117
2025-08-18T02:00:00.103000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_r4kqtao2


### Setup Alice and Bob Node Wallet

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new address to alice so she can sign the funding transaction
alice_wallet_ctx = generate_taproot_ctx()
alice_funding_address = generate_taproot_address(alice_wallet_ctx, index=0, change=False)
print(f"Alice funding address created: {alice_funding_address}")

# Create a new address to bob so he can sign the funding transaction
bob_wallet_ctx = generate_taproot_ctx()
bob_funding_address = generate_taproot_address(bob_wallet_ctx, index=0, change=False)
print(f"Bob funding address created: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
txid = node.sendmany("", {alice_funding_address: 1, bob_funding_address: 1})
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding address created: bcrt1p89ghytrvk4vgnt5xnppyevcdwtvwp80xkq7kewq5v64gjy6ulhgsly3x09
Bob funding address created: bcrt1pkm2t0nxx5e6qrs290zrhlrs04r9kd2qp8xxzqk3h2e88jzhnwx0scfjmsd


## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                              +---------+
    |           |--(1)---  open_channel  ----->|         |
    |           |<-(2)--  accept_channel  -----|         |
    |           |                              |         |
    |   Alice   |--(3)--  funding_created  --->|   Bob   |
    |           |<-(4)--  funding_signed  -----|         |
    |           |                              |         |
    |           |--(5)---  channel_ready  ---->|         |
    |           |<-(6)---  channel_ready  -----|         |
    +-----------+                              +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs: a change output back to Alice and the channel-funding output.

In [4]:
alice_change_address = generate_taproot_address(alice_wallet_ctx, index=0, change=True)
print(f"Alice change address created: {alice_change_address}")

Alice change address created: bcrt1p3kaq8gshnsxgu6569nz4jaqh679glvznpk6y0k4nz392wn7gl3xs4pqnlz
